In [1]:
import pickle

from src.evaluation.evaluator_manager import EvaluatorManager
from src.evaluation.evaluator_manager_do import EvaluatorManager as PairedEvaluatorManager
from src.evaluation.evaluator_manager_triplets import EvaluatorManager as TripletsEvaluatorManager

from src.utils.context import Context

from utils_martina.my_utils import *

In [2]:
import torch
torch.manual_seed(0)

import random
random.seed(0)

import numpy as np
np.random.seed(0)

In [3]:
# Set patient and record (same channels)

# observations = ["chb04_28"]
# observations = ["chb01_03"]
# observations = ["chb01_04"]
# observations = ["chb01_15"]
# observations = ["chb01_16"]
# observations = ["chb01_18"]
# observations = ["chb01_21"]
observations = ["chb01_26"]

create_dataset_json(observations)

In [4]:
logs_path = "..\\..\\explainability\GRETEL-repo\\output\\logs\\"
eval_manager_path = "..\\..\\explainability\GRETEL-repo\\output\\eval_manager\\"

## Removing cache data

In [5]:
remove_cache = True

if remove_cache:
    list_folders = [".\data\cache\datasets", ".\data\cache\explainers", ".\data\cache\oracles"]

    for folder in list_folders:
        for file in os.listdir(folder):
            if file.startswith("GCS-"):  # Checks if the file begins with "GCS-"
                cache_path = os.path.join(folder, file)
                if os.path.isfile(cache_path):
                    os.remove(cache_path)
                    print(f"Removed file: {cache_path}")
                elif os.path.isdir(cache_path):
                    shutil.rmtree(cache_path)
                    print(f"Removed folder: {cache_path}")

Removed file: .\data\cache\datasets\GCS-28e49373646e3a1aa4f78d3b3cf52c7d
Removed folder: .\data\cache\oracles\GCS-28e49373646e3a1aa4f78d3b3cf52c7d


## Run Gretel experiment

In [6]:
# NOTICE: If it crashes, check if there are any OracleTorch files from the future!

In [7]:
path = 'config/GCS-perfect.jsonc'

print(observations)

print(f"Generating context for: {path}")
context = Context.get_context(path)
context.run_number = -1

context.logger.info(f"Executing: {context.config_file} Run: {context.run_number}")
context.logger.info(
    "Creating the evaluation manager......................................................."
)

if 'doe-triplets' in context.conf:
    context.logger.info("Creating the TRIPLET evaluators........................................................")
    eval_manager = TripletsEvaluatorManager(context)
if 'do-pairs' in context.conf:
    context.logger.info("Creating the PAIRED evaluators...............................................................") ### Usiamo questo
    eval_manager = PairedEvaluatorManager(context)
else:
    context.logger.info("Creating the evaluators...............................................................")
    eval_manager = EvaluatorManager(context)

context.logger.info(
    "Evaluating the explainers............................................................."
)

eval_manager.evaluate()

# Additional stuff
print('--------------------------------------')

for i in range(len(eval_manager._evaluators)):
    # context.logger.info(f"graph_ids: {eval_manager._evaluators[i].get_instance_and_counterfactual_ids()}") # NON È STATO SISTEMATO PER CONTEMPLARE IL CASO DI PIU' PAZIENTI
    context.logger.info(f"graph_metrics: {eval_manager._evaluators[i].get_instance_and_counterfactual_graph_metrics()}")

['chb01_26']
Generating context for: config/GCS-perfect.jsonc
2025-05-15 07:50:22,757 | INFO | 22228 - Executing: config/GCS-perfect.jsonc Run: -1
2025-05-15 07:50:22,781 | INFO | 22228 - Creating the evaluation manager.......................................................
2025-05-15 07:50:22,800 | INFO | 22228 - Creating the PAIRED evaluators...............................................................
2025-05-15 07:50:22,877 | INFO | 22228 - Creating: GCS-28e49373646e3a1aa4f78d3b3cf52c7d
2025-05-15 07:50:22,877 | INFO | 22228 - Instantiating: src.dataset.generators.gcs.GCS
2025-05-15 07:50:24,162 | INFO | 22228 - Saved: GCS-28e49373646e3a1aa4f78d3b3cf52c7d
2025-05-15 07:50:24,229 | INFO | 22228 - Created: GCS-28e49373646e3a1aa4f78d3b3cf52c7d
2025-05-15 07:50:24,276 | INFO | 22228 - Creating: PerfectOracle-d4ed22792c9a83c39c971ab18176dee9
2025-05-15 07:50:24,312 | INFO | 22228 - PerfectOracle trained in: 0.0 secs
2025-05-15 07:50:24,366 | INFO | 22228 - Saved: PerfectOracle-d4ed227

In [8]:
# Save eval_manager to file
file_name = get_most_recent_file(logs_path).split('.')[0]

with open(eval_manager_path + file_name + ".pkl", 'wb') as f:
    pickle.dump(eval_manager, f)

## _______________________________________________________________

In [9]:
import datetime

now = datetime.datetime.now()
print("Last full run:", now.strftime("%d/%m/%Y, ore %H:%M"))

Last full run: 15/05/2025, ore 07:50


In [10]:
import winsound

winsound.Beep(440, 500)
winsound.Beep(400, 500)
winsound.Beep(440, 500)

In [11]:
# from src.data_analysis.future.data_analyzer import DataAnalyzer as data_analyzer

# results_path = "output\\results"
# stats_file_path = "output"
# res = data_analyzer.create_aggregated_dataframe(results_path)
# res.to_csv(stats_file_path)
# res